### OpenStreetMap Data Wrangling with SQL
<b>Yuchen Yeh, Oct 2016</b>

### Map Area:
London, United Kingdom

Source: <br>
https://www.openstreetmap.org/relation/65606#map=10/51.4904/-0.0879 <br>
https://mapzen.com/data/metro-extracts/metro/london_england/

### Problems Encountered in the Map

After downloading a small sample size of the London area and running it against a data.py file, there are 4 main problems with the data:

1. Missing user values and uid in nodes (node changeset="120938" id="16713489" lat="51.9539579" lon="-0.1707352" timestamp="2006-09-26T17:25:23Z" version="1")

2. Several data (e.g. ddress and naptan in nodes_tags; source and building in ways_tags) are divided into segments in sub levels “k” tags with the following format: (tag k="addr:city" v="London")

3. Inconsistent street names ( "BLACKWALL TUNNEL NORTHERN APPROACH","Victoria Road South")
	
4. Duplicate key values with different spellings in the node tags (postal_code vs postcode, uri vs url)

5. Incorrect phone number format (441474247750, +44 (1923) 672820)

<b>1. Missing user value and uid </b>

When running the data.py file, KeyError: 'user' was returned. It is due to missing user values in the nodes. To deal with this issue, I used 'try except' to catch the error and set the missing values to be 'na'.

In [ ]:
    if element.tag == 'node':
        for node in NODE_FIELDS:
            try:
                node_attribs[node] = element.attrib[node]
            except Exception:
                node_attribs[node] = 'na'

After wrangling, (node changeset="120938" id="16713489" lat="51.9539579" lon="-0.1707352" timestamp="2006-09-26T17:25:23Z" version="1") is updated as follows:

<table style="width:100%">
  <tr>
    <th>id</th>
    <th>lat</th> 
    <th>lon</th>
    <th>user</th>
    <th>uid</th> 
    <th>version</th>
    <th>changeset</th> 
    <th>timestamp</th>
  </tr>
  <tr>
    <th>76733867</th>
    <th>51.9102213</th> 
    <th>-0.5476516</th>
    <th>na</th>
    <th>na</th> 
    <th>1</th>
    <th>226900</th> 
    <th>2007-10-17T20:43:23Z</th>
  </tr>
</table>

							
							


<b> 2. Data groups are divided into segments in sub levels of “k” tags </b>

"K" tags in the second level store detailed data groups for address, building, etc. In order to better to understand second-level data, I delimited the data by ':' and leave the sub-level in the key tag and the first level of data in type tag.

In [ ]:
elif LOWER_COLON.search(child.attrib["k"]):
     tag_type = child.attrib["k"].split(':',1)[0]
     tag_key = child.attrib["k"].split(':',1)[1]
     tag["key"] = tag_key
     tag["type"] = tag_type
     tag["id"] = element.attrib["id"]
     tag["value"] = child.attrib["v"]

After wrangling, the data looks like:

<table style="width:100%">
  <tr>
    <th>id</th>
    <th>key</th> 
    <th>value</th>
    <th>type</th>
  </tr>
  <tr>
    <th>311835538</th>
    <th>city</th> 
    <th>Maidstone</th>
    <th>addr</th>
  </tr>
 
</table>

In the nodes_tags, naptan data still didn't get separated properly. For example, tag k="naptan:Street" v="STATION APPROACH".
The problem is the original regex of LOWER_COLON would not match the values that the first letter after colon(:) is upper case. Therefore, the solution is to include a match of upper case and lower case.

LOWER_COLON = re.compile(r'^([a-zA-Z]|_)+:([a-zA-Z]|_)+')

Furthermore, some of the key values are segmented into three levels, and they would not be separated properly in the key values. For example, 
tag k="seamark:light:character" v="Q" <br>

Using the following code to pass the third level value into key values and keep the first and second level into the tag type. This ensures we have a similar format in the key tag.

In [ ]:
tag_type = child.attrib["k"].split(':',1)[0]
tag_key = child.attrib["k"].split(':',1)[1]
try:
    tag_extra = tag_key.split(':',1)[1]
    tag_key =tag_extra
    tag_type = child.attrib["k"].split(':',1)[0] +':' +tag_key.split(':',1)[0]
except Exception:
    pass

With this wrangling code, the key values that have three levels look like:

<table style="width:100%">
  <tr>
    <th>id</th>
    <th>key</th> 
    <th>value</th>
    <th>type</th>
  </tr>
  <tr>
    <th>1413554521</th>
    <th>character</th> 
    <th>Q</th>
    <th>seamark:character
</th>
  </tr>
 
</table>

<b>3. Inconsistent street names</b>

Street names were entered in inconsistent letter cases:
    
tag k="naptan:Street" v="BLACKWALL TUNNEL NORTHERN APPROACH"  <br>
tag k="naptan:Street" v="Victoria Road South"

I used function update_street() to update the street values to be capital on the first letter:

STREET = ['street','Street']


def update_street(street):
    street = street.title()
    return street
          
if tag_key in STREET:
    tag["value"]=update_street(child.attrib['v'])
    tag["key"] = tag_key
    tag["id"] = element.attrib["id"]
    tag["type"] = tag_type

<table style="width:100%">
  <tr>
    <th>id</th>
    <th>key</th> 
    <th>value</th>
    <th>type</th>
  </tr>
  <tr>
    <th>4042892954</th>
    <th>Street</th> 
    <th>Blackwall Tunnel Northern Approach</th>
    <th>naptan</th>
  </tr>
 
</table>

<b>4. Duplicate key values with different spellings in the node tags</b>

postal_code vs postcode 

tag k="postal_code" v="ME19 6AJ" <br>
tag k="addr:postcode" v="RH1 1DW"


uri vs url <br>

tag k="uri" v="http://www.rbkc.gov.uk/az/az.asp?orgid=1767" <br>
tag k="url" v="www.architecturalplants.com" 

I updated the values with one version of each key using the following code:

if  tag["key"] == 'postal_code':
    tag["key"] = 'postcode'
if  tag["key"] == 'uri':
    tag["key"] = 'url'

<b> 5. Incorrect phone number format </b>

To standardise phone numbers, I used a function update_phone to clean up the data with a number of regex patterns.

In [ ]:
def update_phone(phone):
    phone = re.sub('\+', r"00", phone) # replace with calling prefix
    phone = re.sub('\(0\)', r"", phone) # replace '(0' between the numbers
    phone = re.sub('\(|\)', r"", phone) # replace '('')' between the numbers
    phone = re.sub('\-', r"", phone) # replace '-'between the numbers
    phone = re.sub('[^0-9]+', '', phone) # only extract numbers
    phone = re.sub('^00440', r"0", phone) # remove calling prefix completely
    phone = re.sub('^0044', r"0", phone) # remove another format of calling prefix completely
    if phone: return phone 

Phone variable are updated as follows: <br>

Before:+44 1844 351621
; After: 01844351621 <br>
Before:+44 (0)1923 840 157
; After:01923840157 <br>
Before:+44118 957 1839
; After:01189571839 <br>

### Data Overview

This section contains basic statistics about the dataset, the SQL queries used to gather them, and some additional ideas about the data in context.

<b>File sizes</b>

In [1]:
import os
folder = 'C:\\sqlite_windows'
folder_size = 0
file_name = ['london_england.osm','sample4.osm','nodes.csv',
             'nodes.tags.csv','ways.csv','ways_nodes.csv','ways_tags.csv']
for (path, dirs, files) in os.walk(folder):
  for file in files:
    if file in file_name:
        filename = os.path.join(path, file)
        folder_size = os.path.getsize(filename)
        print file + "   %0.1f MB" % (folder_size/(1000*1000))    
    else:
        continue

london_england.osm   2558.0 MB
nodes.csv   18.0 MB
sample4.osm   51.0 MB
ways.csv   1.0 MB
ways_nodes.csv   6.0 MB
ways_tags.csv   3.0 MB


<b>Number of nodes</b>

In [2]:
import sqlite3
from pprint import pprint

sqlite_file = 'data.db'

conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.execute( 'SELECT count(*) FROM nodes ')

for row in c:
    print(row[0])


226966


<b>Number of ways </b>

In [3]:
c.execute( 'SELECT count(*) FROM ways')

for row in c:
    print(row[0])

32450


<b>Number of unique tags in nodes</b>

In [4]:
c.execute( 'SELECT count(distinct key) FROM nodes_tags')

for row in c:
    print(row[0])

368


<b>Number of unique tags in ways</b>

In [5]:
c.execute( 'SELECT count(distinct key) FROM ways_tags')

for row in c:
    print(row[0])

474


<b>Top 10 used tags</b>

In [6]:
c.execute( '''SELECT e.key, COUNT(*) as num
FROM (SELECT key FROM nodes_tags UNION ALL SELECT key FROM ways_tags) e
GROUP BY e.key
ORDER BY num DESC
LIMIT 10;''')
for row in c:
    print(row)

(u'building', 15096)
(u'highway', 14272)
(u'name', 10773)
(u'source', 10281)
(u'created_by', 5063)
(u'housenumber', 4597)
(u'street', 4374)
(u'natural', 3018)
(u'maxspeed', 2849)
(u'city', 2383)


<b> Number of unique users </b>

In [7]:
c.execute( '''SELECT COUNT(DISTINCT(e.uid))          
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;''')

for row in c:
    print(row[0])

3381


<b> Top 10 contributing users </b>

In [8]:
c.execute( '''SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10;''')
for row in c:
    print(row)

(u'The Maarssen Mapper', 9555)
(u'Eriks Zelenka', 9152)
(u'TimSC_Data_CC0_To_Andy_Allan', 8939)
(u'ca_hoot', 7240)
(u'Johnmb', 7164)
(u'busdoc', 7129)
(u'Essex_Boy', 5979)
(u'DanGregory', 5309)
(u'c2r', 4631)
(u'Rondon237', 4104)


<b>Top 5 used values in the node tag of highways</b>

In [9]:
c.execute( '''SELECT value, count(*) as num FROM nodes_tags
WHERE key ='highway'
GROUP BY value
ORDER BY num desc
Limit 5;
 ''')

all = c.fetchall()
pprint(all)


[(u'bus_stop', 901),
 (u'crossing', 414),
 (u'traffic_signals', 181),
 (u'turning_circle', 80),
 (u'mini_roundabout', 65)]


<b>Most prevalent tourism type</b>

In [10]:
c.execute( '''SELECT value, count(*) as num FROM nodes_tags
WHERE key ='tourism'
GROUP BY value
ORDER BY num desc
Limit 1;
 ''')

all = c.fetchall()
pprint(all)

[(u'information', 20)]


### Other ideas about the datasets

<b>Posts by user counts</b>

In [11]:
c.execute( '''
    SELECT a.num, count(distinct a.user)
    FROM
    (SELECT e.user, COUNT(*) as num
     FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
     GROUP BY e.user) a
     group by a.num
       ;''')

import pandas as pd

data =[]
for row in c:
    data.append(row)

df = pd.DataFrame(data)  #put the data in a dataframe
df.rename(columns={0:'Post',1:'User_counts'}, inplace=True) #rename columns
count_all = df['User_counts'].sum()
df['Percentage']=df['User_counts']/count_all*100
print df.head(n=10)

   Post  User_counts  Percentage
0     1         1197   35.393258
1     2          450   13.305736
2     3          228    6.741573
3     4          159    4.701360
4     5          113    3.341218
5     6           94    2.779420
6     7           70    2.069781
7     8           66    1.951508
8     9           59    1.744530
9    10           48    1.419279


35% of users only posted once, and there is a total of 73% of users that posted between 1 and 10 posts. 

<b> User contribution</b>

To motivate users to contribute more in the open street projects, we should build on a contribution chart in each map area. For example, 'The Maarssen Mapper' won the top contributor in the London area. This would make people feel that the contribution is recognised. 

However, when more people start to contribute adding data, we may encounter more issues with data quality. Therefore, we should utilise the user contribution system to select champions for each area and have them mentor other contributors and police the data quality.

If we can build in a pre-defined list of key tags (for example addr, phone or post_codes), we could somehow control the variable inputs, improve the data quality and hopefully have a great amount of cleaned data to OpenStreetMap.org.


<b>Conclusion</b>

The data of the London area is imperfect, and the issues related to missing user values, details of data divided into segments in the “k” tags, inconsistent street names and duplicate key values. Most of the data are cleaned up after wrangling. 

It is interesting to see 35% users only posted once, and less than 30% of users contributed more than 10 posts. If we include a contribution chart in each map area to announce top contributor, we may improve user contribution due to better contribution recognition. 

If we can build on a pre-defined list of key tags, we could improve the data quality and have a great amount of cleaned data to OpenStreetMap.org.